In [1]:
import os
import fnmatch
import cv2
import numpy as np
import string
import time

from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization,add, Input, Conv2D, MaxPool2D, Lambda, Bidirectional,GRU,concatenate,Reshape
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
# Check all available devices if GPU is available
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4122501837338607393
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1588555691359846685
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15753608763222600727
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11146722048
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15166062688431528011
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [3]:
data_path="/content/drive/My Drive/cropped_data/cropped_dir/"
label_path="/content/drive/My Drive/cropped_data/annotations.txt"

file=open(label_path)
list=file.read().splitlines()

image_list=[]
label_list=[]
for i in range(len(list)):
  label=(list[i].split("\t")[1])
  image_name=data_path+str(i)+".jpg"
  image_list.append(image_name)
  label_list.append(label)
print(len(label_list))
print(len(image_list))

1740
1740


In [68]:
hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
char_list=[]

char_list=hindi_alphabets
char_list.insert(0,'-')
print(char_list)
def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)
        
    return dig_lst
print(len(char_list))

['-', 'ऀ', 'ँ', 'ं', 'ः', 'ऄ', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऌ', 'ऍ', 'ऎ', 'ए', 'ऐ', 'ऑ', 'ऒ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'ऩ', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ऱ', 'ल', 'ळ', 'ऴ', 'व', 'श', 'ष', 'स', 'ह', 'ऺ', 'ऻ', '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॅ', 'ॆ', 'े', 'ै', 'ॉ', 'ॊ', 'ो', 'ौ', '्', 'ॎ', 'ॏ', 'ॐ', '॑', '॒', '॓', '॔', 'ॕ', 'ॖ', 'ॗ', 'क़', 'ख़', 'ग़', 'ज़', 'ड़', 'ढ़', 'फ़', 'य़', 'ॠ', 'ॡ', 'ॢ', 'ॣ', '।', '॥', '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', '॰', 'ॱ', 'ॲ', 'ॳ', 'ॴ', 'ॵ', 'ॶ', 'ॷ', 'ॸ', 'ॹ', 'ॺ', 'ॻ', 'ॼ', 'ॽ', 'ॾ', 'ॿ']
129


In [69]:
mylist=[i+"+"+j for i,j in zip(image_list,label_list)]
print(mylist)
from sklearn.model_selection import train_test_split
train,test = train_test_split(mylist, test_size=0.15,random_state=11)
print(len(train), len(test))

['/content/drive/My Drive/cropped_data/cropped_dir/0.jpg+जल', '/content/drive/My Drive/cropped_data/cropped_dir/1.jpg+शोध', '/content/drive/My Drive/cropped_data/cropped_dir/2.jpg+न', '/content/drive/My Drive/cropped_data/cropped_dir/3.jpg+संयंत्र', '/content/drive/My Drive/cropped_data/cropped_dir/4.jpg+बिना', '/content/drive/My Drive/cropped_data/cropped_dir/5.jpg+आज्ञा', '/content/drive/My Drive/cropped_data/cropped_dir/6.jpg+प्रवेश', '/content/drive/My Drive/cropped_data/cropped_dir/7.jpg+निषेध', '/content/drive/My Drive/cropped_data/cropped_dir/8.jpg+नगर', '/content/drive/My Drive/cropped_data/cropped_dir/9.jpg+पालिका', '/content/drive/My Drive/cropped_data/cropped_dir/10.jpg+परिषद', '/content/drive/My Drive/cropped_data/cropped_dir/11.jpg+इटारसी', '/content/drive/My Drive/cropped_data/cropped_dir/12.jpg+जिला', '/content/drive/My Drive/cropped_data/cropped_dir/13.jpg+होशंगाबाद', '/content/drive/My Drive/cropped_data/cropped_dir/14.jpg+मप्र', '/content/drive/My Drive/cropped_data/c

In [70]:
import cv2
import matplotlib.pyplot as plt
training_img = []
training_txt = []
train_input_length = []
train_label_length = []
orig_txt = []
 
max_label_len = 0

 
for i in range(len(train)):
  image = train[i].split("+")[0]
  label=train[i].split("+")[1]
  image = plt.imread(image)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  img=gray
  h,w=img.shape
  if h<w and h>70:
    img=cv2.resize(img,(128,32))
    img = np.expand_dims(img , axis = 2)
    image_norm=img/255
    if len(label)>max_label_len:
      max_label_len=len(label)
    orig_txt.append(label)   
    train_label_length.append(len(label))
    train_input_length.append(31)
    training_img.append(image_norm)
    training_txt.append(encode_to_labels(label)) 

valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []

for i in range(len(test)):
  image = test[i].split("+")[0]
  label=test[i].split("+")[1]
  image = plt.imread(image)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  img=gray
  h,w=img.shape
  if h<w and h>70:
  
    img=cv2.resize(img,(128,32))
    img = np.expand_dims(img , axis = 2)
    image_norm=img/255
    if len(label)>max_label_len:
      max_label_len=len(label)

    valid_orig_txt.append(label)   
    valid_label_length.append(len(label))
    valid_input_length.append(31)
    valid_img.append(image_norm)
    valid_txt.append(encode_to_labels(label))

In [71]:
print(len(training_img),len(valid_img))
print(max_label_len)

995 171
14


In [72]:
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value =0)
valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = 0)

In [73]:
print(train_padded_txt[3])
print(training_txt[3])
print(orig_txt[3])

[22 65  0  0  0  0  0  0  0  0  0  0  0  0]
[22, 65]
की


In [74]:
inputs = Input(shape=(32,128,1))

conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
batch_norm_1 = BatchNormalization()(conv_1)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(batch_norm_1)

conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
batch_norm_2= BatchNormalization()(conv_2)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(batch_norm_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
batch_norm_3 = BatchNormalization()(conv_3)
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
batch_norm_4 = BatchNormalization()(conv_4)
pool_4 = MaxPool2D(pool_size=(2, 1))(batch_norm_4)
 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
batch_norm_5 = BatchNormalization()(conv_5)
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)


blstm_1 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(blstm_1)

#GRU
'''
gru_1 = GRU(128, return_sequences=True, kernel_initializer='he_normal', name='gru1',dropout = 0.1)(squeezed)  # (None, 32, 512)
gru_1b = GRU(128, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b',dropout = 0.1)(squeezed)
reversed_gru_1b = Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (gru_1b)

gru1_merged = add([gru_1, reversed_gru_1b])  # (None, 32, 512)
gru1_merged = BatchNormalization()(gru1_merged)
    
gru_2 = GRU(128, return_sequences=True, kernel_initializer='he_normal', name='gru2',dropout = 0.1)(gru1_merged)
gru_2b = GRU(128, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b',dropout = 0.1)(gru1_merged)
reversed_gru_2b= Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (gru_2b)

gru2_merged = concatenate([gru_2, reversed_gru_2b])  # (None, 32, 1024)
gru2_merged = BatchNormalization()(gru2_merged)
'''
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

# model to be used at test time
act_model = Model(inputs, outputs)

In [75]:
act_model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 128, 1)]      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 32, 128, 64)       640       
_________________________________________________________________
batch_normalization_18 (Batc (None, 32, 128, 64)       256       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 16, 64, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 16, 64, 128)       73856     
_________________________________________________________________
batch_normalization_19 (Batc (None, 16, 64, 128)       512       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 8, 32, 128)      

In [76]:
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
 
 
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

#model to be used at training time
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

In [77]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')
 
filepath="best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

In [78]:
training_img1 = np.array(training_img)
train_input_length1 = np.array(train_input_length)
train_label_length1 = np.array(train_label_length)

valid_img1 = np.array(valid_img)
valid_input_length1 = np.array(valid_input_length)
valid_label_length1 = np.array(valid_label_length)

In [ ]:
batch_size = 16
epochs = 10
model.fit(x=[training_img1, train_padded_txt, train_input_length1, train_label_length1],
          y=np.zeros(len(training_img1)), 
          batch_size=batch_size, epochs = epochs, 
          validation_data = ([valid_img1, valid_padded_txt, valid_input_length1, valid_label_length1], [np.zeros(len(valid_img1))]),
          verbose = 1 )

In [86]:
weight_path="best_model.hdf5"
act_model.save_weights(weight_path)

In [87]:
act_model.load_weights('best_model.hdf5')

prediction = act_model.predict(valid_img1[:30])
 
# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
 
# see the results
i = 0
for x in out:
    print("original_text =  ", valid_orig_txt[i])
    print("predicted text = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(char_list[int(p)], end = '')       
    print('\n')
    i+=1

original_text =   छात्रावास
predicted text = छात्रावास

original_text =   समिति
predicted text = समित

original_text =   पुस्तकालय
predicted text = पुस्तालर

original_text =   विवाह
predicted text = लिफ्ाद

original_text =   कर्मचारी
predicted text = कर्मचारी

original_text =   प
predicted text = प

original_text =   सी
predicted text = सी

original_text =   जीवन
predicted text = जौतन

original_text =   विश्राम
predicted text = विव्याम

original_text =   ध्यान
predicted text = स्ान

original_text =   उपयोग
predicted text = जाोंज

original_text =   न
predicted text = कर

original_text =   स्कूल
predicted text = स्वत

original_text =   स्टेशन
predicted text = सेलल

original_text =   इटारसी
predicted text = इटारसी

original_text =   एस
predicted text = एुरण

original_text =   यात्री
predicted text = जाी

original_text =   रेल
predicted text = रेल

original_text =   द्वितीय
predicted text = विजीय

original_text =   रेल
predicted text = रेलल

original_text =   सप्ताह
predicted text = सराद

